# Google Research - Identify Contrails to Reduce Global Warming
This is the notebook that uses a UNET model for inference for the "Google Research - Identify Contrails to Reduce Global Warming" competition on Kaggle. It ranked 765/954 with dice score 0.59090.

The dataset used for training was made by kaggler Shashwat Raman.

# Imports

In [ ]:
from pathlib import Path
import os
import random
import math
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import albumentations as A
import torch.nn.functional as F

from PIL import Image
from tqdm.notebook import tqdm
from transformers import get_cosine_schedule_with_warmup
from tqdm.auto import tqdm

In [ ]:
import sys

sys.path.append("/kaggle/input/pretrained-models-pytorch")
sys.path.append("/kaggle/input/efficientnet-pytorch")
sys.path.append("/kaggle/input/smp-github/segmentation_models.pytorch-master")

import segmentation_models_pytorch as smp

print(f"Segmentation Models version: {smp.__version__}")

In [ ]:
class Config:
    train = True
    
    num_epochs = 10
    
    thr = 0.02
    
    # No mask doesn't count as class
    num_classes = 1
    
    batch_size = 32
    seed = 42
    
    encoder = 'efficientnet-b0'
    pretrained = True
    
    # If "weights = 'imagenet'," because imagenet was not imported, it must be downloaded from internet, but Kaggle notebook doesn't get internet
    # access in submission which is why we must use "weights = None."
    weights = None
    
    # Class names are listed here. Class names themselves aren't actually used in the code. Only the 
    # length of this list is.
    classes = ['contrail']
    activation = None
    in_chans = 3
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    image_size = 256
    warmup = 0
    lr = 3e-4
    
    trained_model_state = '/kaggle/input/trained-model-state-2/epoch-19.pth'

    
class Paths:
    data_root = '/kaggle/input/google-research-identify-contrails-reduce-global-warming'
    contrails = '/kaggle/input/contrails-images-ash-color/contrails/'
    test_root = '/kaggle/input/google-research-identify-contrails-reduce-global-warming/test/'

In [ ]:
class ContrailsDataset(torch.utils.data.Dataset):
    def __init__(self, df, train=True):
        
        self.df = df
        self.trn = train
    
    # Handles reading from a directory under test (i.e. reads band files)
    # Note: Only bands 11, 14, and 15 are used to create the final tensor
    def read_record(self, directory):
        
        # Stores numpy arrays in a dictionary
        record_data = {}
        for x in [
            "band_11", 
            "band_14", 
            "band_15"
        ]:

            record_data[x] = np.load(os.path.join(directory, x + ".npy"))

        # Returns dictionary mapping band name (i.e. "band_11") to numpy array for that band file
        # (i.e. band_11.npy converted to numpy array)
        return record_data

    def normalize_range(self, data, bounds):
        """Maps data to the range [0, 1]."""
        return (data - bounds[0]) / (bounds[1] - bounds[0])
    
    # This is the function responsible for taking multiple bands as input, and returning a single tensor
    # "False color" = ": color in an image (such as a photograph) of an object that does not actually 
    # appear in the object but is used to enhance, contrast, or distinguish details."
    def get_false_color(self, record_data):
        _T11_BOUNDS = (243, 303)
        _CLOUD_TOP_TDIFF_BOUNDS = (-4, 5)
        _TDIFF_BOUNDS = (-4, 2)
        
        N_TIMES_BEFORE = 4
        
        # "Combines" the 'band_15', 'band_14', and 'band_11' tensors (each are 3D, H x W x Time step)
        # Note: r, g, and b are still H x W x Time step (256 x 256 x 8)
        r = self.normalize_range(record_data["band_15"] - record_data["band_14"], _TDIFF_BOUNDS)
        g = self.normalize_range(record_data["band_14"] - record_data["band_11"], _CLOUD_TOP_TDIFF_BOUNDS)
        b = self.normalize_range(record_data["band_14"], _T11_BOUNDS)

        # Combines 3 x (H x W x T or 256 x 256 x 8) tensors into one H x W x C x T (256 x 256 x 3 x 8)
        false_color = np.clip(np.stack([r, g, b], axis=2), 0, 1)
        
        # Slice includes all dimensions 1 - 3, but only the 'N_TIMES_BEFORE'th 4th dimension element
        img = false_color[..., N_TIMES_BEFORE]

        return img
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        con_path = row.path
        data = self.read_record(con_path)    

        img = self.get_false_color(data)
        
        img = torch.tensor(img)
        
        # Changes tensor shape from H x W x C (256 x 256 x 3) to C x H x W (3 x 256 x 256)
        img = img.permute(2, 0, 1)
        
        return img.float()
    
    def __len__(self):
        return len(self.df)

In [ ]:
test_ids = os.listdir(Paths.test_root)
test_df = pd.DataFrame(test_ids, columns=['record_id'])
test_df['path'] = Paths.test_root + test_df['record_id'].astype(str)

test_ds = ContrailsDataset(test_df, False)

test_dl = DataLoader(test_ds, batch_size=Config.batch_size, num_workers=2)

In [ ]:
class UNet(nn.Module):
    def __init__(self, cfg):
        super(UNet, self).__init__()
        
        self.cfg = cfg
        self.training = True
        
        self.model = smp.Unet(
            encoder_name=cfg.encoder, 
            encoder_weights=cfg.weights, 
            decoder_use_batchnorm=True,
            classes=len(cfg.classes), 
            activation=cfg.activation,
        )
        
        self.loss_fn = smp.losses.DiceLoss(mode='binary')
    
    def forward(self, imgs):
        
        x = imgs
        logits = self.model(x)
        
        return {"logits": logits.sigmoid()}

In [ ]:
model = UNet(Config).to(Config.device)
model.load_state_dict(torch.load(Config.trained_model_state, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))

In [ ]:
model.eval()
torch.set_grad_enabled(False)

val_data = defaultdict(list)

test_data_loader = tqdm(enumerate(test_dl), total=len(test_dl), desc='test')

for ind, X in test_data_loader:
    X = X.to(Config.device)
    
    output = model(X)
    
    for key, val in output.items():
        val_data[key]+=[val]
        
for key, val in output.items():
    value = val_data[key]
    
    if len(value[0].shape)==0:
        val_data[key] = torch.stack(value)
    else:
        val_data[key] = torch.cat(value).cpu().detach().numpy()

In [ ]:
def rle_encode(x, fg_val=1):
    """
    Args:
        x:  numpy array of shape (height, width), 1 - mask, 0 - background
    Returns: run length encoding as list
    """

    dots = np.where(
        x.T.flatten() == fg_val)[0]  # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def list_to_string(x):
    """
    Converts list to a string representation
    Empty list returns '-'
    """
    if x: # non-empty list
        s = str(x).replace("[", "").replace("]", "").replace(",", "")
    else:
        s = '-'
    return s

In [ ]:
submission = pd.read_csv(Paths.data_root+'/sample_submission.csv', index_col='record_id')

display(submission)

for i, prediction in enumerate(val_data['logits']):
    rec = test_df['record_id'][i]
    mask = (prediction[0]>Config.thr).astype(np.float32)
    submission.loc[int(rec), 'encoded_pixels'] = list_to_string(rle_encode(mask))

submission.head()

In [ ]:
submission.to_csv('submission.csv')